# Set Up 2023 Data

In [ ]:
import json
import pandas as pd

filepath = "Zillow-ChapelHill-Spring2023-18k-Properties.json"

# First, Load the JSON file and convery it to a pandas data frame
with open(filepath) as f:
  data2023 = json.load(f)
data2023.keys()


df_2023 = pd.DataFrame(data2023)

# Display the first few rows of the DataFrame
print(df_2023.head())


# Set up 2025 Data

In [ ]:

filepath0 = 'Zillow-March2025-dataset_part0.json'
filepath1 = 'Zillow-March2025-dataset_part1.json'
filepath2 = 'Zillow-March2025-dataset_part2.json'
filepath3 = 'Zillow-March2025-dataset_part3.json'

# Load the JSON files and convert them to pandas data frames
with open(filepath0, encoding='utf-8') as f:
  data0 = json.load(f)
with open(filepath1, encoding='utf-8') as f:
  data1 = json.load(f)
with open(filepath2, encoding='utf-8') as f:
  data2 = json.load(f)
with open(filepath3, encoding='utf-8') as f:
  data3 = json.load(f)
# Combine the data from all files into a single DataFrame
df0 = pd.DataFrame(data0)
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)
df3 = pd.DataFrame(data3)
df_2025 = pd.concat([df0, df1, df2, df3], ignore_index=True)


# 2025 Walkable Locations

reduce data to locations that are <1 mile from center of campus 

In [ ]:
from geopy.distance import geodesic

# Define key campus locations (e.g., The Pit, Franklin Street)
unc_coordinates = (35.9106, -79.0472)
maximum_distance = 1  # in miles

# Calculate walking/biking distance for each property
def calculate_distance_to_location(row, location_coords):
    property_coords = (row['latitude'], row['longitude'])
    return geodesic(property_coords, location_coords).miles

# Remove rows with missing or invalid latitude/longitude
df_2025 = df_2025.dropna(subset=['latitude', 'longitude'])

# Calculate distance from campus for each property
df_2025['distance_from_campus'] = df_2025.apply(
    lambda row: calculate_distance_to_location(row, unc_coordinates), axis=1
)

# Filter for properties within 5 miles of campus
df_2025_nearby = df_2025[df_2025['distance_from_campus'] <= maximum_distance]

df_2025 = df_2025_nearby

# Display the filtered properties
print(df_2025[['streetAddress', 'city', 'state', 'price', 'bedrooms', 'bathrooms', 'distance_from_campus', 'daysOnZillow']])



# Property Type Filter

Focus on:
- Condos, townhomes, single family homes
- Units with 2–3 bedrooms (for possible roommates)





### Posible properties
- 'apartment' 
- 'single_family' 
- 'townhouse' 
- 'multi_family' 
- 'condo'
- 'home_type_unknown' 
- 'lot'

In [ ]:

home_types = ['single_family', 'townhouse', 'condo']

# Clean up the 'homeType' column
df_2025.loc[:, 'homeType'] = df_2025['homeType'].str.lower() # this just makes sure all home types are in lower case
df_2025_expected_home_type = df_2025[
    (df_2025_nearby['homeType'].isin(home_types)) &
    (df_2025_nearby['bedrooms'].between(2, 3))
]

df_2025 = df_2025_expected_home_type
# Display the filtered properties
print(df_2025[['streetAddress', 'city', 'homeType', 'price', 'bedrooms', 'bathrooms', 'distance_from_campus', 'daysOnZillow']])

# Time on Market / Turnover
- Look at average days on market for each property
- Higher turnover may signal student-friendly properties
- Low turnover could signal quiet, long-term-owner zones ideal for parents seeking stability

In [ ]:
max_time_on_market = 60 # days

# Filter for properties that have been on the market for less than 1000 days
df_2025_high_turnover = df_2025[df_2025['daysOnZillow'] <= max_time_on_market]

df_2025 = df_2025_high_turnover

# Display the filtered properties
print(df_2025[['streetAddress', 'city', 'homeType', 'price', 'bedrooms', 'bathrooms', 'distance_from_campus', 'daysOnZillow']])


# Property Features
- In-unit laundry (major preference)
- Dedicated parking
- Safe, well-lit paths or sidewalks to campus
    

# Transit Routes

- Evaluate proximity to Chapel Hill Transit stops
- Bonus points for routes with direct connection to campus


# Appreciation Rate

- Compare what properties went up 20% between 2023 and 2025 sets and filter

In [ ]:
# Clean up price and zpid columns
df_2023['price'] = pd.to_numeric(df_2023['price'], errors='coerce')
df_2025['price'] = pd.to_numeric(df_2025['price'], errors='coerce')
df_2023['zpid'] = pd.to_numeric(df_2023['zpid'], errors='coerce')
df_2025['zpid'] = pd.to_numeric(df_2025['zpid'], errors='coerce')

# Merge 2023 and 2025 data on 'zpid'
merged_df = pd.merge(
    df_2025[['zpid', 'price', 'streetAddress']],  # keep zpid, price, streetAddress from 2025
    df_2023[['zpid', 'price']],  # keep zpid and price from 2023
    on='zpid',
    suffixes=('_2025', '_2023')
)

# Calculate percentage price increase
merged_df['price_increase_percentage'] = ((merged_df['price_2025'] - merged_df['price_2023']) / merged_df['price_2023']) * 100

# Filter properties with price appreciation ≥ 20%
filtered_properties = merged_df[merged_df['price_increase_percentage'] >= 20]

# Show the result
print(filtered_properties[['zpid', 'streetAddress', 'price_2023', 'price_2025', 'price_increase_percentage']])
